Importing all the Required Libraries

In [2]:
import numpy as np
import pandas as pd
import string 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn .svm import SVC
from sklearn.linear_model import LogisticRegression

Loading the Dataset Collected

In [60]:
dataframe = pd.read_csv('dataset.csv')
dataframe.head()

,category,rating,LABEL,text_
0,Home_and_Kitchen_5,5,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5,CG,Very nice set. Good quality. We have had the s...


Creating a new column named Length for the length of each review

In [61]:
dataframe.dropna(inplace=True)
dataframe['Length'] = dataframe['text_'].apply(len)
print(dataframe[dataframe['LABEL']=='OR'][['text_','Length']].sort_values(by='Length', ascending = False).head().iloc[0].text_)
dataframe.head()

WEAK ON CURRENT SCIENCE.
After seeing it twice, I agree with much (but not all) of the positive five star reviews. Out of respect for those who READ reviews, I'll not repeat everything that I like about the presentation. I found the goofy oversize earrings, hairdo, and facial hair arrangement of Daniel Vitalis, (described as a "Wild Food Expert") distracting. UGH. Ditto for David Wolfe, who had an extremely goofy wild hairdo. On the other hand, Jon Gabriel, described as an "author and weight loss expert" was nicely groomed and a good presenter. His story of personal transformation of a fellow of over 400 pounds (whew) to becoming a jock of normal weight was inspiring. Christiane Northrup preserves her rank as one of America's cutest doctors. A really nice looking woman! Presentations by Dr. Mercola, Jason Vale, Kris Carr, Alejandro Junger were fine. It was disappointing to have Jamie Oliver (so popular in the UK) give Baby Cow Growth Fluid a pass with unscientific but popular ideas on 

,category,rating,LABEL,text_,Length
0,Home_and_Kitchen_5,5,CG,"Love this! Well made, sturdy, and very comfor...",75
1,Home_and_Kitchen_5,5,CG,"love it, a great upgrade from the original. I...",80
2,Home_and_Kitchen_5,5,CG,This pillow saved my back. I love the look and...,67
3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i...",81
4,Home_and_Kitchen_5,5,CG,Very nice set. Good quality. We have had the s...,85


Removing Punctuations and Stop Words

In [62]:
def convertmyTxt(rv):
    np = [c for c in rv if c not in string.punctuation]
    np = ''.join(np)
    return [w for w in np.split() if w.lower() not in stopwords.words('english')]

In [63]:
dataframe['Stop Words'] = dataframe.apply(lambda x: convertmyTxt(x['text_']), axis=1)

In [7]:
'''def porter_stem(val):
    stemmer = PorterStemmer()
    return [stemmer.stem(word).lower() for word in val]'''

In [11]:
#dataframe['Porter'] = dataframe.apply(lambda x: porter_stem(x['Stop Words']), axis=1)

In [64]:
def rejoin_words(tokenized_column):
    return ( " ".join(tokenized_column))
dataframe['all_text'] = dataframe.apply(lambda x: rejoin_words(x['Stop Words']), axis=1)

In [65]:
dataframe.head()

,category,rating,LABEL,text_,Length,Stop Words,all_text
0,Home_and_Kitchen_5,5,CG,"Love this! Well made, sturdy, and very comfor...",75,"[Love, Well, made, sturdy, comfortable, love, ...",Love Well made sturdy comfortable love itVery ...
1,Home_and_Kitchen_5,5,CG,"love it, a great upgrade from the original. I...",80,"[love, great, upgrade, original, Ive, mine, co...",love great upgrade original Ive mine couple years
2,Home_and_Kitchen_5,5,CG,This pillow saved my back. I love the look and...,67,"[pillow, saved, back, love, look, feel, pillow]",pillow saved back love look feel pillow
3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i...",81,"[Missing, information, use, great, product, pr...",Missing information use great product price
4,Home_and_Kitchen_5,5,CG,Very nice set. Good quality. We have had the s...,85,"[nice, set, Good, quality, set, two, months]",nice set Good quality set two months


In [66]:
dataframe.to_csv('FakeReview01.csv')

=================================================================================================

In [67]:
dataframe = pd.read_csv('FakeReview01.csv')
dataframe.head()

,Unnamed: 0,category,rating,LABEL,text_,Length,Stop Words,all_text
0,0,Home_and_Kitchen_5,5,CG,"Love this! Well made, sturdy, and very comfor...",75,"['Love', 'Well', 'made', 'sturdy', 'comfortabl...",Love Well made sturdy comfortable love itVery ...
1,1,Home_and_Kitchen_5,5,CG,"love it, a great upgrade from the original. I...",80,"['love', 'great', 'upgrade', 'original', 'Ive'...",love great upgrade original Ive mine couple years
2,2,Home_and_Kitchen_5,5,CG,This pillow saved my back. I love the look and...,67,"['pillow', 'saved', 'back', 'love', 'look', 'f...",pillow saved back love look feel pillow
3,3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i...",81,"['Missing', 'information', 'use', 'great', 'pr...",Missing information use great product price
4,4,Home_and_Kitchen_5,5,CG,Very nice set. Good quality. We have had the s...,85,"['nice', 'set', 'Good', 'quality', 'set', 'two...",nice set Good quality set two months


Removing NAN values from dataframe

In [69]:
dataframe = dataframe.dropna(subset=["LABEL", "all_text"])  

Splitting the data to Training and Testing as 75:25

In [79]:
x_train, x_test, y_train, y_test = train_test_split(dataframe['all_text'],dataframe['LABEL'], test_size= 0.25,random_state=42 )
print("X Train \n",x_train,"Y Train\n", y_train,"X Test\n", x_test,"Y Test\n", y_test)

X Train 
 6156     bota bag awsome perfect size strong strap real...
5711     Shirt large basically tshirt design comfortabl...
11086    First way turn volume turn volume phone recogn...
20401    dog interested bone little texture hard piece ...
16804    Works great Craftsman tool set long keep hands...
                               ...                        
6265     usually shoot multiple weapons thought would l...
11284    Supper bright litium battery whats love light ...
38159    usually drive 35 miles one way purchase orderi...
860      Great playroom multiple children fits bit ever...
15795    Looks great made quality material problem come...
Name: all_text, Length: 30323, dtype: object Y Train
 6156     OR
5711     CG
11086    CG
20401    OR
16804    CG
         ..
6265     CG
11284    OR
38159    OR
860      OR
15795    CG
Name: LABEL, Length: 30323, dtype: object X Test
 27957    Eva find way escape abusive mother meets hands...
39755    second Glamorise love look keep bag w

Creating A Pipeline to fit the Training dataset

In [80]:
pip = Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',RandomForestClassifier())
])

Fitting The 75% Training dataset to the Pipeline

In [81]:
pip.fit(x_train, y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

Predicting The Label Class for Test Dataset

In [82]:
rfc = pip.predict(x_test)
print(rfc)

['CG' 'CG' 'OR' ... 'CG' 'OR' 'CG']


Taking Review From User and Predicting its Target Value

In [83]:
test = pd.Series(["Got the 256GB version at 21k and it is really worth the price. Camera can be slightly better but battery backup and other features are really good and worthy for the price paid.","I ordered F 19 pro two times one delivered in May and another in Aug. Only after receiving the second F19 Pro , I realized that first phone packet did not have the the earbuds headphone. I request that I be supplied the earbuds deficient in the first order. Also while confirming the dispatch, pls mention list of supplied items.","Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty","The Oppo F19 Pro+ 5G is a solid mid-range offering that delivers impressive performance and an attractive design"])
check = input("Enter the Text")
checkk = pd.Series([check])
pred = pip.predict(test)
for x in pred:
    print(x)

OR
OR
CG
CG


Finding The Accuracy and Confusion Matrix of The Model

In [84]:
print('Accuracy of the model: ',str(np.round(accuracy_score(y_test,rfc)*100,2)) + '%')
listt = confusion_matrix(y_test,rfc)
print("TP :",listt[0][0])
print("FP :",listt[0][1])
print("FN :",listt[1][0])
print("TN :",listt[1][1])
#85.25%
#85.12%

Accuracy of the model:  85.12%
TP : 4505
FP : 536
FN : 968
TN : 4099
